In [2]:
import numpy as np
import tensorflow as tf

c:\Users\PRITAM\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))


In [45]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Parameters
    ----------
    z : array_like
        A scalar or numpy array of any size.

    Returns
    -------
     g : array_like
         sigmoid(z)
    """
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

#### Dataset

In [5]:
X,Y = load_coffee_data()

In [6]:
print(X.shape, Y.shape)

(200, 2) (200, 1)


Normalize data

In [27]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


Numpy Model (Forward Prop in NumPy)

In [70]:
def my_dense(a_in, W, b, g):
    """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
      g    activation function (e.g. sigmoid, relu..)
    Returns
      a_out (ndarray (j,))  : j units|
    """

    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):
        w = W[:,j]
        z = np.dot(w, a_in) + b[j]
        a_out[j] = g(z)
    return(a_out)  

The following cell builds a two-layer neural network utilizing the my_dense subroutine above.

In [46]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x, W1, b1, sigmoid)
    a2 = my_dense(x, W2, b2, sigmoid)
    return a2

We can copy trained weights and biases from the previous lab in Tensorflow.

In [47]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

Let's start by writing a routine similar to Tensorflow's model.predict(). This will take a matrix *X* with all *m* examples in the rows and make a prediction by running the model.

In [48]:
def my_predict(X, W1, b1, W2, b2):
    m = X.shape[0]
    p = np.zeros(m,1)
    for i in range(m):
        p[i,0] = my_sequential(X[i], W1, b1, W2, b2)
    return(p)  

In [182]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize

In [171]:
X_tst[0].shape

(2,)

In [79]:
m = X_tstn.shape[0]
p = np.zeros((m,1))


In [195]:
for i in range(m):
    p[i,0] = X_tstn[i]
print(p)
    

ValueError: setting an array element with a sequence.

In [161]:
X_tstn[0]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ -5.468438, -11.614221], dtype=float32)>

In [43]:
arr= np.random.rand(10, 2)
y = np.array([1,0,1,1,0,0,1,1,0,0])
units = arr.shape[1]

In [39]:
np.zeros(units)

array([0., 0.])

In [173]:
X_test = np.array([0.14, 0.325])   
# X_test = norm_l(X_test)

In [174]:
X_test.shape

(2,)

In [178]:
a_in = X_test
W = W1_tmp
b = b1_tmp
g = sigmoid
    

units = W.shape[1]
a_out = np.zeros(units)
for j in range(units):               
    w = W[:,j]                                    
    z = np.dot(w, a_in) + b[j]      
    a_out[j] = g(z)               
    print(a_out)

[1.50713484e-05 0.00000000e+00 0.00000000e+00]
[1.50713484e-05 8.99889290e-06 0.00000000e+00]
[1.50713484e-05 8.99889290e-06 9.98128702e-01]


In [177]:
W1_tmp.T.shape, X_test.shape

((3, 2), (2,))

In [186]:
a1 = my_dense(X_tstn[1],  W1_tmp, b1_tmp, sigmoid)
a2 = my_dense(a1, W2_tmp, b2_tmp, sigmoid)

In [187]:
a1

array([2.56118858e-03, 7.16559110e-15, 1.00000000e+00])

In [188]:
a2

array([3.2897871e-08])

In [ ]:
X_tstn

In [194]:
3.2897871e-08 * 10

3.2897871e-07